# Parameters 

In [1]:
fileName = '4PpKk.hdf5'
dsNameStates = '4PpKk-fullStateBool'
dsNameWDLs = '4PpKk-Wdl'
convertStates = False
# fileName = 'AllStates_intVec.hdf5'
nPi = 4
nPa = 2
nWPa = 1
input_shape = (4,8,8)
batch_size = 4096
epochs = 300
dataDiv = 1
num_classes = 5

multiGPU = False
whichGPU = 0

# Imports

In [2]:
# ### To select which GPU to use, import these before importing Keras or Tensorflow
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either &quot;0&quot; or &quot;1&quot;
# os.environ["CUDA_VISIBLE_DEVICES"]="0"  
 
# # Do other imports now...

In [3]:

if(multiGPU):
    from keras.utils.training_utils import multi_gpu_model
else:
    import os
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    # The GPU id to use, usually either "0" or "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(whichGPU)

# from __future__ import print_function
import keras
# from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
# import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import h5py
import numpy as np

Using TensorFlow backend.


## Analysis
```
3PKk-Wdl-Buffered == 3PKk-Wdl-Seq

4PpKk-Wdl == 4PpKk-Wdl-Buffered != 4PpKk-Wdl-retry

5PPpKk-Wdl != 5PPpKk-Wdl-Buffered
    from 50.000 then buffered != 10, but wdl == 10
```
  
### WDL score count
#### 3PKk-WDL-Seq
[0, 0, 125024, 0, 124960]

# Functions for vector to full state conversion AND MORE!

In [4]:
def sq2hnit(sq):
    col = sq%8
    row = (sq - col)//8
    return col,row

# 0: pawns
# 1: kings
def vecSt2fullSt(vecSt, nPi, nPa, nWPa):
    fullSt = np.zeros((4,8,8), dtype = 'bool')
    for i in range(nPi - 2):
        sq = vecSt[i]
        col,row = sq2hnit(sq)
        if i < nWPa:
            fullSt[0][row][col] = True
        else:
            fullSt[1][row][col] = True
    col,row = sq2hnit(vecSt[-2])
    fullSt[2][row][col] = True
    col,row = sq2hnit(vecSt[-1])
    fullSt[3][row][col] = True
    return fullSt 

def vecSt2fullSt_8x8x2(vecSt, nPi, nPa, nWPa):
    fullSt = np.zeros((8,8,2), dtype = 'int8')
    for i in range(nPi - 2):
        sq = vecSt[i]
        col,row = sq2hnit(sq)
        if i < nWPa:
            fullSt[row][col][0] = 1
        else:
            fullSt[row][col][0] = -1
    col,row = sq2hnit(vecSt[-2])
    fullSt[row][col][1] = 1
    col,row = sq2hnit(vecSt[-1])
    fullSt[row][col][1] = -1
    return fullSt 
# nPi = 3
# nPa = 1
# nWPa = 1
# vecSt2fullSt(d3[0], nPi, nPa, nWPa)
# nPi = 5
# nPa = 3
# nWPa = 2
# vecSt2fullSt(d5[0], nPi, nPa, nWPa)
    

# count nr of each score instance
# wdlCounter placeholders: [-2, -1, 0, 1 ,2]

def wdlCountingMachine(ds):
    wdlCounter = [0,0,0,0,0]
    l = len(ds)
    i = 0
    intv = l//100
    for wdl in ds:
        i += 1
        if i%intv == 0:
            sys.stdout.write(str((i*100)//l) + " percentage")
            sys.stdout.write('\r')
            sys.stdout.flush()
        wdlCounter[wdl[0] + 2] += 1
    print(wdlCounter)
    return wdlCounter
# wdlCountingMachine(d3t)

# Load data

In [5]:
# load datasets
f = h5py.File(fileName, 'a')
d = f[dsNameStates]
dt = f[dsNameWDLs]
# d5 = f['5PPpKk']
l = len(d)

if convertStates:
    X = np.array([vecSt2fullSt(vecSt,nPi, nPa, nWPa) for vecSt in d[:l//dataDiv]])
else:
    X = d[:l//dataDiv]

y = dt[:l//dataDiv]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

del X
del y

print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

X_train shape: (10216846, 4, 8, 8)
y_train shape: (10216846, 1)
X_test shape: (5032178, 4, 8, 8)
y_test shape: (5032178, 1)
10216846 train samples
5032178 test samples


In [6]:
# del f['4PpKk-fullState']
# f = h5py.File(fileName,'a')
# d = f['4PpKk']
# X = np.array([vecSt2fullSt(vecSt,nPi, nPa, nWPa) for vecSt in d[:l//dataDiv]])
# f.create_dataset('4PpKk-fullState', data = X)
# for i in f:print(i)
# f.close()


# Create Keras model

In [7]:
# import keras.backend as K
# K.set_floatx('float16')
# K.set_epsilon(1e-4) #default is 1e-7
# K.set_floatx('float32')
# K.set_epsilon(1e-7) #default is 1e-7

model = Sequential()
filterDivider = 2
l1filters = 64//filterDivider
l2filters = 96//filterDivider
l3filters = 256//filterDivider
print(l1filters)
print(l2filters)
print(l3filters)

model.add(Conv2D(l1filters, kernel_size=(3, 3),
                 padding='valid',
                 activation='relu',
                 data_format = "channels_first",
#                  kernel_initializer = 
                 input_shape=input_shape))
model.add(Conv2D(l2filters, kernel_size=(3, 3),
                 padding='valid',
                 activation='relu'))
model.add(Conv2D(l3filters, kernel_size=(3, 3),
                 padding='valid',
                 activation='relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
if multiGPU: 
    model = keras.utils.multi_gpu_model(model, gpus=2)
model.summary()
model.save_weights('tempWeights.hdf5')


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


# model.save_weights('tempWeights.hdf5')
# model.load_weights('tempWeights.hdf5')

32
48
128
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 6, 6)          1184      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 4, 48)         2640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 2, 128)        55424     
_________________________________________________________________
flatten_1 (Flatten)          (None, 7168)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 35845     
Total params: 95,093
Trainable params: 95,093
Non-trainable params: 0
_________________________________________________________________


# Train

In [ ]:
model.fit(X_train, y_train,
      batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 10216846 samples, validate on 5032178 samples
Epoch 1/300
10216846/10216846 [==============================] - 105s 10us/step - loss: 0.2987 - acc: 0.8723 - val_loss: 0.1751 - val_acc: 0.9254
Epoch 2/300
10216846/10216846 [==============================] - 104s 10us/step - loss: 0.1550 - acc: 0.9351 - val_loss: 0.1368 - val_acc: 0.9431
Epoch 3/300
10216846/10216846 [==============================] - 105s 10us/step - loss: 0.1274 - acc: 0.9471 - val_loss: 0.1201 - val_acc: 0.9494
Epoch 4/300
10216846/10216846 [==============================] - 105s 10us/step - loss: 0.1131 - acc: 0.9534 - val_loss: 0.1174 - val_acc: 0.9521
Epoch 5/300
10216846/10216846 [==============================] - 105s 10us/step - loss: 0.1038 - acc: 0.9574 - val_loss: 0.1043 - val_acc: 0.9558
Epoch 6/300
10216846/10216846 [==============================] - 105s 10us/step - loss: 0.0969 - acc: 0.9604 - val_loss: 0.0941 - val_acc: 0.9614
Epoch 7/300
10216846/10216846 [==============================] - 105s

10216846/10216846 [==============================] - 105s 10us/step - loss: 0.0498 - acc: 0.9804 - val_loss: 0.0616 - val_acc: 0.9758
Epoch 57/300
10216846/10216846 [==============================] - 104s 10us/step - loss: 0.0497 - acc: 0.9805 - val_loss: 0.0576 - val_acc: 0.9775
Epoch 58/300
10216846/10216846 [==============================] - 104s 10us/step - loss: 0.0495 - acc: 0.9805 - val_loss: 0.0577 - val_acc: 0.9774
Epoch 59/300
10216846/10216846 [==============================] - 104s 10us/step - loss: 0.0493 - acc: 0.9806 - val_loss: 0.0629 - val_acc: 0.9756
Epoch 60/300
10216846/10216846 [==============================] - 104s 10us/step - loss: 0.0492 - acc: 0.9806 - val_loss: 0.0593 - val_acc: 0.9769
Epoch 61/300
10216846/10216846 [==============================] - 105s 10us/step - loss: 0.0490 - acc: 0.9808 - val_loss: 0.0589 - val_acc: 0.9769
Epoch 62/300
10216846/10216846 [==============================] - 104s 10us/step - loss: 0.0489 - acc: 0.9808 - val_loss: 0.0562 - 

In [ ]:
# for i in range(7):
#     batch_size = 64 * 2 ** i
#     print("batch_size: ", batch_size)
#     model.fit(X_train, y_train,
#           batch_size=batch_size,
#               epochs=1,
#               verbose=1,
#               validation_data=(X_test, y_test))

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model.save_weights('4PpKk_1_weights.hdf5')